# Import Libraries

In [1]:
import pandas as pd
import numpy as np

## Load the dataset.

In [10]:
flights=pd.read_csv("D:\\Flask_tutorial\\mytrip\\model\\flights.csv")
hotels=pd.read_csv("D:\\Flask_tutorial\\mytrip\\model\\hotels.csv")
users=pd.read_csv("D:\\Flask_tutorial\\mytrip\\model\\users.csv")

In [11]:
flights.head()

,travelCode,userCode,from,to,flightType,price,time,distance,agency,date
0,0,0,Recife (PE),Florianopolis (SC),firstClass,1434.38,1.76,676.53,FlyingDrops,09/26/2019
1,0,0,Florianopolis (SC),Recife (PE),firstClass,1292.29,1.76,676.53,FlyingDrops,09/30/2019
2,1,0,Brasilia (DF),Florianopolis (SC),firstClass,1487.52,1.66,637.56,CloudFy,10/03/2019
3,1,0,Florianopolis (SC),Brasilia (DF),firstClass,1127.36,1.66,637.56,CloudFy,10/04/2019
4,2,0,Aracaju (SE),Salvador (BH),firstClass,1684.05,2.16,830.86,CloudFy,10/10/2019


In [12]:
hotels.head()

,travelCode,userCode,name,place,days,price,total,date
0,0,0,Hotel A,Florianopolis (SC),4,313.02,1252.08,09/26/2019
1,2,0,Hotel K,Salvador (BH),2,263.41,526.82,10/10/2019
2,7,0,Hotel K,Salvador (BH),3,263.41,790.23,11/14/2019
3,11,0,Hotel K,Salvador (BH),4,263.41,1053.64,12/12/2019
4,13,0,Hotel A,Florianopolis (SC),1,313.02,313.02,12/26/2019


In [13]:
users.head()

,code,company,name,gender,age
0,0,4You,Roy Braun,male,21
1,1,4You,Joseph Holsten,male,37
2,2,4You,Wilma Mcinnis,female,48
3,3,4You,Paula Daniel,female,23
4,4,4You,Patricia Carson,female,44


In [14]:
flights.columns

Index(['travelCode', 'userCode', 'from', 'to', 'flightType', 'price', 'time',
       'distance', 'agency', 'date'],
      dtype='object')

In [15]:
hotels.columns

Index(['travelCode', 'userCode', 'name', 'place', 'days', 'price', 'total',
       'date'],
      dtype='object')

In [16]:
users.columns

Index(['code', 'company', 'name', 'gender', 'age'], dtype='object')

In [18]:
print("flights:",flights.shape)
print("users:",users.shape)
print("hotels:",hotels.shape)


flights: (271888, 10)
users: (1340, 5)
hotels: (40552, 8)


In [38]:
import pandas as pd

def recommend_hotels(flights_file, hotels_file, users_file):
    # Load datasets
    flights_df = pd.read_csv(flights_file)
    hotels_df = pd.read_csv(hotels_file)
    users_df = pd.read_csv(users_file)
    
    # Ensure necessary columns exist
    required_flight_cols = {'travelCode', 'userCode', 'to'}
    required_hotel_cols = {'travelCode', 'userCode', 'name', 'place', 'days', 'price', 'total', 'date'}
    
    if not required_flight_cols.issubset(flights_df.columns):
        raise ValueError("Missing required columns in flights dataset")
    if not required_hotel_cols.issubset(hotels_df.columns):
        raise ValueError("Missing required columns in hotels dataset")
    
    # Merge datasets on userCode and travelCode
    merged_df = pd.merge(flights_df, hotels_df, on=['travelCode', 'userCode'], how='inner', suffixes=('_flight', '_hotel'))
    
    # Debug: Print column names to check for suffixes
    print("Merged DataFrame columns:", merged_df.columns)
    
    # Filter hotels based on the city where the user landed
    recommended_hotels = merged_df[merged_df['to'] == merged_df['place']]
    
    # Ensure correct column names after merging
    # recommendations = recommended_hotels[['to', 'place', 'name', 'days', 'price_hotel', 'date_hotel']].copy()
    recommendations = recommended_hotels[[ 'place', 'name', 'days', 'price_hotel', 'date_hotel']].copy()# removing the column 'to'
    recommendations.rename(columns={'name': 'hotel_name', 'price_hotel': 'price', 'date_hotel': 'date'}, inplace=True)

    
    return recommendations

In [39]:
# Example usage
if __name__ == "__main__":
    flights_path = "D:\\Flask_tutorial\\mytrip\\model\\flights.csv"
    hotels_path = "D:\\Flask_tutorial\\mytrip\\model\\hotels.csv"
    users_path = "D:\\Flask_tutorial\\mytrip\\model\\users.csv"
    recommendations = recommend_hotels(flights_path, hotels_path, users_path)
    print(recommendations.head())


Merged DataFrame columns: Index(['travelCode', 'userCode', 'from', 'to', 'flightType', 'price_flight',
       'time', 'distance', 'agency', 'date_flight', 'name', 'place', 'days',
       'price_hotel', 'total', 'date_hotel'],
      dtype='object')
                place hotel_name  days   price        date
0  Florianopolis (SC)    Hotel A     4  313.02  09/26/2019
2       Salvador (BH)    Hotel K     2  263.41  10/10/2019
4       Salvador (BH)    Hotel K     3  263.41  11/14/2019
6       Salvador (BH)    Hotel K     4  263.41  12/12/2019
8  Florianopolis (SC)    Hotel A     1  313.02  12/26/2019


In [40]:
print(recommendations)

                    place hotel_name  days   price        date
0      Florianopolis (SC)    Hotel A     4  313.02  09/26/2019
2           Salvador (BH)    Hotel K     2  263.41  10/10/2019
4           Salvador (BH)    Hotel K     3  263.41  11/14/2019
6           Salvador (BH)    Hotel K     4  263.41  12/12/2019
8      Florianopolis (SC)    Hotel A     1  313.02  12/26/2019
...                   ...        ...   ...     ...         ...
81094       Brasilia (DF)   Hotel BP     3  247.62  06/18/2020
81096       Brasilia (DF)   Hotel BP     1  247.62  06/25/2020
81098   Campo Grande (MS)   Hotel BW     3   60.39  07/02/2020
81100   Campo Grande (MS)   Hotel BW     3   60.39  07/09/2020
81102          Natal (RN)   Hotel BD     4  242.88  07/16/2020

[40552 rows x 5 columns]


recommend hotel using ML:

In [1]:
pip install scikit-surprise


  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build scikit-surprise


  error: subprocess-exited-with-error
  
  × Building wheel for scikit-surprise (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [115 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-312\surprise
      copying surprise\accuracy.py -> build\lib.win-amd64-cpython-312\surprise
      copying surprise\builtin_datasets.py -> build\lib.win-amd64-cpython-312\surprise
      copying surprise\dataset.py -> build\lib.win-amd64-cpython-312\surprise
      copying surprise\dump.py -> build\lib.win-amd64-cpython-312\surprise
      copying surprise\reader.py -> build\lib.win-amd64-cpython-312\surprise
      copying surprise\trainset.py -> build\lib.win-amd64-cpython-312\surprise
      copying surprise\utils.py -> build\lib.win-amd64-cpython-312\surprise
      copying surprise\__init__.py -> build\lib.win-amd64-cpython-312\surprise
      copying surprise\__main__.py -> build\lib.win-amd64-cpython-312\surpris

In [41]:
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def recommend_hotels_using_flight_data(flights_file, hotels_file):
    # Load data
    flights_df = pd.read_csv(flights_file)
    hotels_df = pd.read_csv(hotels_file)
    
    # Merge the two datasets on travelCode and userCode
    merged_df = pd.merge(flights_df, hotels_df, on=['travelCode', 'userCode'], how='inner', suffixes=('_flight', '_hotel'))
    
    # Feature Engineering for content-based filtering
    # Use 'place' from hotels and 'to' from flights to create a similarity metric
    # For now, using a simple content-based filtering approach using cosine similarity
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(merged_df['place'] + " " + merged_df['to'])
    
    # Calculate cosine similarity between hotels based on 'place' and 'to'
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    # Function to get hotel recommendations based on destination similarity
    def get_similar_hotels(hotel_id, cosine_sim=cosine_sim):
        # Get pairwise similarity scores for the given hotel
        sim_scores = list(enumerate(cosine_sim[hotel_id]))
        
        # Sort hotels based on similarity
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Get top 10 similar hotels (excluding the first one, which is the hotel itself)
        sim_scores = sim_scores[1:11]
        
        # Get the hotel indices
        hotel_indices = [i[0] for i in sim_scores]
        
        return merged_df.iloc[hotel_indices]
    
    # Recommend hotels for a user based on flight destination and hotel features
    user_id = 1  # Example: Get recommendations for user with userCode 1
    
    # Filter the merged dataset for the specific user
    user_hotels = merged_df[merged_df['userCode'] == user_id]
    
    # Get recommendations based on the user's last visited hotel (for simplicity, taking the first one)
    hotel_id = user_hotels.iloc[0]['hotel_id']  # Assuming each hotel has a unique ID
    recommendations = get_similar_hotels(hotel_id)
    
    return recommendations[['userCode', 'hotel_id', 'place', 'name', 'price', 'date']]

# Example usage
flights_file = 'flights.csv'
hotels_file = 'hotels.csv'
recommendations = recommend_hotels_using_flight_data(flights_file, hotels_file)
print(recommendations)


ModuleNotFoundError: No module named 'surprise'

In [2]:
im

NameError: name 'im' is not defined

In [3]:
import pandas as pd

In [5]:
rent_df=pd.read_csv("D:\\Flask_tutorial\\mytrip\\model\\rent_a_car.csv")

In [6]:
rent_df.head()

,travelCode,code,pickupLocation,dropoffLocation,carType,rentalAgency,pickupDateTime,dropoffDateTime,rentalDuration,price,totalDistance,fuelPolicy,bookingStatus
0,126191,195,Aliciabury,Lopezbury,Sedan,Enterprise,2025-01-19 02:43:34,2025-01-24 02:43:34,5,284.56,371,Prepaid,Cancelled
1,18474,1086,Edwardsborough,North Daniel,SUV,Sixt,2025-01-01 07:31:29,2025-01-03 07:31:29,2,176.83,461,Full-to-Full,Pending
2,90427,90,West Danielside,Riverafort,Sedan,Avis,2025-01-02 22:37:32,2025-01-05 22:37:32,3,236.79,383,Prepaid,Confirmed
3,123934,75,North Samantha,North Joseph,Luxury,Sixt,2025-01-16 18:57:49,2025-01-21 18:57:49,5,781.64,294,Partial,Pending
4,98809,300,Cobbview,Amandaburgh,Luxury,Sixt,2025-01-20 22:53:00,2025-01-25 22:53:00,5,597.22,488,Partial,Cancelled


In [7]:
rent_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   travelCode       10000 non-null  int64  
 1   code             10000 non-null  int64  
 2   pickupLocation   10000 non-null  object 
 3   dropoffLocation  10000 non-null  object 
 4   carType          10000 non-null  object 
 5   rentalAgency     10000 non-null  object 
 6   pickupDateTime   10000 non-null  object 
 7   dropoffDateTime  10000 non-null  object 
 8   rentalDuration   10000 non-null  int64  
 9   price            10000 non-null  float64
 10  totalDistance    10000 non-null  int64  
 11  fuelPolicy       10000 non-null  object 
 12  bookingStatus    10000 non-null  object 
dtypes: float64(1), int64(4), object(8)
memory usage: 1015.8+ KB
